### RFC

### import modules and configure notebook

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import LocalOutlierFactor

pd.set_option('max.rows', None)
pd.set_option('max.columns', None)
%matplotlib inline

C:\Users\tom\Anaconda3\lib\site-packages\sklearn\utils\validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'


### Load variables stored by data_preproccessing notebook

In [2]:
%store -r train_data_formodel
%store -r test_data
%store -r my_data
%store -r uniques
%store -r best_feats

### configurations
* save_plots -> boolean
* scale -> True|False if set to True then features scaled to all have mean value 0 and standard deviation 1

In [3]:
save_plots = True

#the data should be scaled because LOF model uses KNN
scale = True

### counts of instances in all classes before oversampling

In [4]:
train_data_formodel['class'].value_counts()

4     105
15    100
16     61
0      53
11     45
13     36
14     36
2      36
6      30
7      30
10     30
5      27
8      27
1      24
12     21
3      18
9      17
Name: class, dtype: int64

### The class column is stored as the variable y 

In [5]:
y_train = np.array(train_data_formodel['class'])

### The variables identified as best by feature selection are used as features

In [6]:
train_data_feats = train_data_formodel[best_feats]
test_data_feats = test_data[best_feats]
test_data_identifiers = test_data['Analysis']

### features are standardised

In [7]:
if scale:
    my_scaler = StandardScaler()
    X_test_scaled = np.array(my_scaler.fit_transform(test_data_feats))
    X_train = np.array(my_scaler.fit_transform(train_data_feats))
else:
    X_test = np.array(test_data_feats)
    X_train = np.array(train_data_feats)

### the dimensions of the class and features are checked

In [8]:
print(X_train.shape)
print(X_test_scaled.shape)
print(y_train.shape)

(696, 15)
(432, 15)
(696,)


### Local Outlier Factor model is fitted on data that will be used for training the final classifier model
* The model is used for the purpose of novelty detection

In [9]:
lof = LocalOutlierFactor(novelty=True,n_neighbors=25 )
#set novelty to true, this is suitable because we are trying to assess whether the artefcats come from the same distribution of the geological samples, refer to sklearn documentation for more details 

### print details of model

In [10]:
print(lof)

LocalOutlierFactor(algorithm='auto', contamination='legacy', leaf_size=30,
          metric='minkowski', metric_params=None, n_jobs=None,
          n_neighbors=25, novelty=True, p=2)


In [11]:
#train model on geological samples
lof.fit(X_train)

C:\Users\tom\Anaconda3\lib\site-packages\sklearn\neighbors\lof.py:236: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarning)
C:\Users\tom\Anaconda3\lib\site-packages\sklearn\neighbors\base.py:442: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  old_joblib = LooseVersion(joblib_version) < LooseVersion('0.12')
C:\Users\tom\Anaconda3\lib\site-packages\sklearn\neighbors\base.py:442: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  old_joblib = LooseVersion(joblib_version) < LooseVersion('0.12')


LocalOutlierFactor(algorithm='auto', contamination='legacy', leaf_size=30,
          metric='minkowski', metric_params=None, n_jobs=None,
          n_neighbors=25, novelty=True, p=2)

### Local Outlier Factor model predicts whether the artefact samples are inliers or outliers 

* inlier = 1
* outier = -1

In [12]:
predictions = lof.predict(X_test_scaled, )

C:\Users\tom\Anaconda3\lib\site-packages\sklearn\neighbors\base.py:442: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  old_joblib = LooseVersion(joblib_version) < LooseVersion('0.12')
C:\Users\tom\Anaconda3\lib\site-packages\sklearn\neighbors\base.py:442: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  old_joblib = LooseVersion(joblib_version) < LooseVersion('0.12')


In [13]:
pd.Series(predictions).value_counts()

 1    301
-1    131
dtype: int64

In [14]:
nInliers = list(pd.Series(predictions).value_counts())[0]
nOutliers = list(pd.Series(predictions).value_counts())[1]

In [15]:
print('there are {0} inliers and {1} outliers, so the proportion of inliers is {2}'.format(nInliers, nOutliers, (nInliers)/(X_test_scaled.shape[0])))

there are 301 inliers and 131 outliers, so the proportion of inliers is 0.6967592592592593


### column encoding inlier status is added to artefact-sample dataset 

In [16]:
X_test_df = pd.DataFrame(data = np.array(test_data_feats), columns = test_data_feats.columns.values)
X_test_labeled_df = pd.concat([test_data_identifiers.reset_index(drop = True), X_test_df, pd.Series(predictions)], axis =1).rename(columns = {0:'inlierLabel'})

In [17]:
X_test_labeled_df.head()

,Analysis,Zr90,Ba137,Sr88,Ge72,Cr52,S33,U238,Al27,B11,Mg24,Nd146,Sc45,K39,Pr141,Li7,inlierLabel
0,005_AB_1,0.24,1.06,3.563843,0.80,6.31,487.95,0.210000,306.53,58.850000,25.138079,0.17,1.00,217.05,0.04,19.35,-1
1,006_AB_1,0.30,0.58,1.180000,0.64,4.78,486.15,0.510000,413.97,85.030000,41.100000,0.17,1.25,353.39,0.06,2.61,-1
2,007_AB_1,0.25,0.49,1.250000,0.67,4.62,480.47,0.242569,364.50,80.040000,44.560000,0.24,1.12,329.78,0.07,2.29,1
3,008_AB_2,0.24,4.74,4.350000,0.71,4.46,482.84,0.030000,139.26,70.736204,10.690000,0.63,1.46,90.89,0.20,6.86,-1
4,009_AB_2,0.30,4.22,3.850000,0.98,4.65,506.40,0.060000,180.19,70.736204,18.670000,0.57,1.47,122.66,0.18,8.45,1


### dataset is stored

In [18]:
%store X_test_labeled_df

Stored 'X_test_labeled_df' (DataFrame)
